# VNNLib Benchmark Visualization
Demonstrate ACT's VNNLib loader with ACAS Xu network visualization

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np

## VNNLib ACAS Xu Perturbation Visualization

This section demonstrates:
- Loading ACAS Xu networks from VNNLib benchmarks
- Parsing VNNLIB specifications for input bounds
- Visualizing input specifications as bar charts
- Testing network behavior on sample points
- Interpreting ACAS Xu collision avoidance outputs

In [ ]:
# ACT VNNLib: Visualize ACAS Xu Specs and Test Robustness
import sys
import os

# Setup ACT paths
act_root = os.path.dirname(os.path.dirname(os.path.abspath('__file__')))
if act_root not in sys.path:
    sys.path.insert(0, act_root)

import torch
import matplotlib.pyplot as plt
import numpy as np
from act.front_end.vnnlib_loader.data_model_loader import load_vnnlib_pair
from act.front_end.vnnlib_loader.create_specs import VNNLibSpecCreator

# Load ACAS Xu model and spec
print("Loading ACAS Xu network and specification...\n")
try:
    # Try to load an ACAS Xu example
    vnnlib_pair = load_vnnlib_pair(
        category="acasxu_2023",
        onnx_model="ACASXU_run2a_1_2_batch_2000.onnx",
        vnnlib_spec="prop_1.vnnlib"
    )
    
    model = vnnlib_pair['model']
    input_tensor = vnnlib_pair['input_tensor']
    vnnlib_metadata = vnnlib_pair['vnnlib_metadata']
    
    print(f"✓ Model loaded: {type(model)}")
    print(f"✓ Input tensor shape: {input_tensor.shape}")
    print(f"✓ VNNLib metadata: {vnnlib_metadata['num_inputs']} inputs, {vnnlib_metadata['num_outputs']} outputs")
    print(f"✓ Property type: {vnnlib_metadata['property_type']}")
    
    # ACAS Xu has 5 inputs (normalized relative position/velocity)
    # Extract bounds from vnnlib_metadata
    input_names = ['ρ (range)', 'θ (angle)', 'ψ (heading)', 'v_own', 'v_int']
    input_bounds = vnnlib_metadata['input_bounds']
    
    # Convert bounds dict to arrays
    num_inputs = vnnlib_metadata['num_inputs']
    lb = np.array([input_bounds[i][0] if i in input_bounds else 0.0 for i in range(num_inputs)])
    ub = np.array([input_bounds[i][1] if i in input_bounds else 0.0 for i in range(num_inputs)])
    center = (lb + ub) / 2
    
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))
    fig.suptitle('ACAS Xu Input Specification Visualization', fontsize=16, fontweight='bold')
    
    # Plot 1: Input bounds as bar chart
    x = np.arange(len(input_names))
    width = 0.35
    
    axes[0].bar(x - width/2, lb, width, label='Lower Bound', color='blue', alpha=0.7)
    axes[0].bar(x + width/2, ub, width, label='Upper Bound', color='red', alpha=0.7)
    axes[0].plot(x, center, 'ko-', label='Center', linewidth=2, markersize=8)
    
    axes[0].set_xlabel('Input Variables', fontsize=12)
    axes[0].set_ylabel('Normalized Value', fontsize=12)
    axes[0].set_title('Input Bounds for ACAS Xu Network', fontweight='bold')
    axes[0].set_xticks(x)
    axes[0].set_xticklabels(input_names, rotation=45, ha='right')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)
    
    # Plot 2: Input range widths
    ranges = ub - lb
    axes[1].barh(input_names, ranges, color='green', alpha=0.7)
    axes[1].set_xlabel('Range Width (ub - lb)', fontsize=12)
    axes[1].set_title('Input Perturbation Ranges', fontweight='bold')
    axes[1].grid(True, alpha=0.3, axis='x')
    
    plt.tight_layout()
    plt.show()
    
    # Test with 3 sample points: lower bound, center, upper bound
    print("\n" + "="*70)
    print("Testing Network on Sample Points")
    print("="*70)
    
    test_points = {
        'Lower Bound': torch.from_numpy(lb).float().unsqueeze(0),
        'Center': torch.from_numpy(center).float().unsqueeze(0),
        'Upper Bound': torch.from_numpy(ub).float().unsqueeze(0)
    }
    
    model.eval()
    with torch.no_grad():
        for name, point in test_points.items():
            output = model(point)
            action = output.argmax(dim=1).item()
            action_names = ['Clear-of-Conflict', 'Weak Left', 'Weak Right', 
                           'Strong Left', 'Strong Right']
            print(f"\n{name}:")
            print(f"  Input: {point.numpy().flatten()}")
            print(f"  Output: {output.numpy().flatten()}")
            print(f"  Recommended Action: {action_names[action]}")
    
    print("\n" + "="*70)
    print(f"✓ Tested ACAS Xu network on {len(test_points)} sample points")
    print(f"  Input dimension: {len(lb)}")
    print(f"  Output dimension: {output.shape[1]}")
    
except Exception as e:
    print(f"⚠ Could not load ACAS Xu example: {e}")
    print("\nNote: ACAS Xu models require:")
    print("  1. ONNX model file (.onnx)")
    print("  2. VNNLib specification file (.vnnlib)")
    print("  3. Proper paths in data/vnnlib/ directory")
    print("\nFalling back to CIFAR10 visualization...")
    
    # Fallback: Show CIFAR10 as alternative
    cifar_dataset = torchvision.datasets.CIFAR10(
        root='./data', train=False, download=True,
        transform=transforms.ToTensor()
    )
    
    class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer', 
                   'dog', 'frog', 'horse', 'ship', 'truck']
    
    fig, axes = plt.subplots(2, 5, figsize=(12, 5))
    fig.suptitle('CIFAR10 Dataset - First 10 Images (Fallback)', fontsize=16)
    
    for idx, ax in enumerate(axes.flat):
        img, label = cifar_dataset[idx]
        img = img.numpy().transpose(1, 2, 0)
        ax.imshow(img)
        ax.set_title(f'{class_names[label]}')
        ax.axis('off')
    
    plt.tight_layout()
    plt.show()
    
    print(f"\nCIFAR10 Dataset size: {len(cifar_dataset)}")
    print(f"Image shape: {cifar_dataset[0][0].shape}")

## Understanding VNNLib Specifications

VNNLib specifications use SMT-LIB format to define:
- Input constraints (box bounds, L∞ perturbations)
- Output properties (safety, robustness)
- Verification tasks for standardized benchmarks

In [1]:
print("="*70)
print("VNNLib Benchmark Approach")
print("="*70)
print("""
✓ **STANDARDIZED BENCHMARKS**: Pre-defined verification tasks
  - ACAS Xu: Aircraft collision avoidance (5 inputs, 5 outputs)
  - VNN-COMP: Competition benchmarks for fair comparison
  - Multiple domains: MNIST, CIFAR, control systems

✓ **SPECIFICATIONS IN SMT-LIB**: Formal constraint definitions
  - Input bounds: (assert (>= X_0 0.6)) (assert (<= X_0 0.68))
  - Output properties: Safety conditions, robustness requirements
  - Property types: Local vs. global specifications

✓ **VERIFICATION WORKFLOW**: 
  1. Load ONNX model and VNNLIB spec
  2. Parse input bounds from SMT-LIB assertions
  3. Convert to internal format (tensors, metadata)
  4. Run verification with specified property
  5. Return SAT/UNSAT/UNKNOWN result

This approach is ideal for:
  • Research paper benchmarking
  • Competition participation (VNN-COMP)
  • Reproducible verification results
  • Standardized tool comparison

CONTRAST WITH TORCHVISION:
  • VNNLib: Fixed specs, standardized benchmarks, SMT-LIB format
  • TorchVision: Custom specs, flexible datasets, Python API
  • Both valid: Use VNNLib for benchmarks, TorchVision for custom tasks
""")

VNNLib Benchmark Approach

✓ **STANDARDIZED BENCHMARKS**: Pre-defined verification tasks
  - ACAS Xu: Aircraft collision avoidance (5 inputs, 5 outputs)
  - VNN-COMP: Competition benchmarks for fair comparison
  - Multiple domains: MNIST, CIFAR, control systems

✓ **SPECIFICATIONS IN SMT-LIB**: Formal constraint definitions
  - Input bounds: (assert (>= X_0 0.6)) (assert (<= X_0 0.68))
  - Output properties: Safety conditions, robustness requirements
  - Property types: Local vs. global specifications

✓ **VERIFICATION WORKFLOW**: 
  1. Load ONNX model and VNNLIB spec
  2. Parse input bounds from SMT-LIB assertions
  3. Convert to internal format (tensors, metadata)
  4. Run verification with specified property
  5. Return SAT/UNSAT/UNKNOWN result

This approach is ideal for:
  • Research paper benchmarking
  • Competition participation (VNN-COMP)
  • Reproducible verification results
  • Standardized tool comparison

CONTRAST WITH TORCHVISION:
  • VNNLib: Fixed specs, standardized ben